In [5]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd 
import numpy as np 

# Load
df = pd.read_csv("historical_data.csv", index_col=0)



In [4]:
df.head()

,province,carcolour,hp,gender,age,lic_years,pastclaims,claims,exp_years
id,,,,,,,,,
1,MP,Blue,74,0,18,0,0,0,0.005476
2,GP,White,283,0,81,63,1,0,0.616016
3,WC,Green,122,0,54,35,1,1,0.919918
4,WC,Black,172,0,39,21,1,0,0.361396
5,LIM,White,161,0,54,35,0,0,1.160849


In [92]:
df_2 = pd.get_dummies(df, columns=["province", "carcolour"], drop_first=True)
bool_cols = df_2.select_dtypes(include=["bool"]).columns
df_2[bool_cols] = df_2[bool_cols].astype(int)


df_3 = df.drop(columns=["province"])
df_3 = pd.get_dummies(df_3, columns=["carcolour"], drop_first=True)
bool_cols = df_3.select_dtypes(include=["bool"]).columns
df_3[bool_cols] = df_3[bool_cols].astype(int)

df_4 = df.drop(columns=["carcolour", "province"])

df_5 = df_4.drop(columns=["hp"])

X = df_5.drop(columns=["claims"])
y = df_4["claims"] 

In [93]:
X.head()
# normalize continuous variables
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#X[["hp", "lic_years", "age"]] = scaler.fit_transform(X[["hp", "lic_years", "age"]])
X.head()

,gender,age,lic_years,pastclaims,exp_years
id,,,,,
1,0,18,0,0,0.005476
2,0,81,63,1,0.616016
3,0,54,35,1,0.919918
4,0,39,21,1,0.361396
5,0,54,35,0,1.160849


In [94]:
y.head()

id
1    0
2    0
3    1
4    0
5    0
Name: claims, dtype: int64

In [95]:
#X_new = X.drop(columns=["hp", "lic_years"])
X_new = X 
data = pd.concat([X, y], axis=1)
data.head()

,gender,age,lic_years,pastclaims,exp_years,claims
id,,,,,,
1,0,18,0,0,0.005476,0
2,0,81,63,1,0.616016,0
3,0,54,35,1,0.919918,1
4,0,39,21,1,0.361396,0
5,0,54,35,0,1.160849,0


In [96]:
model = smf.poisson("claims ~ " + " + ".join(X_new.columns), data=data).fit()

Optimization terminated successfully.
         Current function value: 0.405872
         Iterations 6


In [97]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:                 claims   No. Observations:                80000
Model:                        Poisson   Df Residuals:                    79994
Method:                           MLE   Df Model:                            5
Date:                Tue, 18 Nov 2025   Pseudo R-squ.:                 0.04271
Time:                        10:48:46   Log-Likelihood:                -32470.
converged:                       True   LL-Null:                       -33918.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.1329      0.283     -7.549      0.000      -2.687      -1.579
gender         0.1110      0.019      5.872      0.000       0.074       0.148
age           -0.0263      0.015     -1.730      0.084      -0.056       0.003
lic_years      0.0104      0.015      0.680      0.497      -0.019       0.040
pastclaims     0.0797      0.009      9.361      0.000       0.063       0.096
exp_years      1.0026      0.023     43.153      0.000       0.957       1.048
==============================================================================
"""

In [73]:
model = sm.GLM(
    y,
    X_new,
    family=sm.families.Poisson(link=sm.families.links.sqrt())
).fit()

c:\Users\Armandt\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The sqrt link alias is deprecated. Use Sqrt instead. The sqrt link alias will be removed after the 0.15.0 release.
  warnings.warn(
c:\Users\Armandt\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:308: DomainWarning: The sqrt link function does not respect the domain of the Poisson family.
  warnings.warn((f"The {type(family.link).__name__} link function "


In [74]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 claims   No. Observations:                80000
Model:                            GLM   Df Residuals:                    79988
Model Family:                 Poisson   Df Model:                           11
Link Function:                   sqrt   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -32634.
Date:                Tue, 18 Nov 2025   Deviance:                       44257.
Time:                        10:32:59   Pearson chi2:                 1.27e+05
No. Iterations:                   100   Pseudo R-squ. (CS):            0.03159
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
hp                   0.0227      0.002     12.822      0.000       0.019       0.026
gender               0.0392      0.003     11.435      0.000       0.032       0.046
age                 -0.0228      0.058     -0.394      0.694      -0.136       0.091
lic_years           -0.0292      0.058     -0.504      0.614      -0.143       0.084
pastclaims           0.0249      0.002     15.399      0.000       0.022       0.028
exp_years            0.2883      0.004     74.097      0.000       0.281       0.296
carcolour_Blue       0.0665      0.007      9.204      0.000       0.052       0.081
carcolour_Green      0.0775      0.007     10.904      0.000       0.064       0.091
carcolour_Red        0.0758      0.005     14.286      0.000       0.065       0.086
carcolour_Silver     0.0850      0.006     13.618      0.000       0.073       0.097
carcolour_White      0.0746      0.005     16.553      0.000       0.066       0.083
carcolour_Yellow     0.0708      0.008      8.843      0.000       0.055       0.086
====================================================================================
"""

In [ ]:
# One-hot encode categoricals
df = pd.get_dummies(df, columns=["province", "carcolour"], drop_first=True)
bool_cols = df.select_dtypes(include=["bool"]).columns
df[bool_cols] = df[bool_cols].astype(int)

# Response variable
y = df["claims"]

# Predictors (exclude 'id', 'claims', 'exp_years')
X = df.drop(columns=["id", "claims", "exp_years"])

# Add constant
X = sm.add_constant(X)

# Fit model with exposure offset
model = sm.GLM(
    y,
    X,
    family=sm.families.Poisson(),
    offset=np.log(df["exp_years"])
).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 claims   No. Observations:                80000
Model:                            GLM   Df Residuals:                    79980
Model Family:                 Poisson   Df Model:                           19
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -32288.
Date:                Mon, 17 Nov 2025   Deviance:                       43565.
Time:                        08:41:10   Pearson chi2:                 8.09e+04
No. Iterations:                     6   Pseudo R-squ. (CS):            0.01824
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -1.4256      0.285  

In [15]:
new_df = pd.read_csv("new_policies.csv")

In [16]:
new_df.head()


,id,province,carcolour,hp,gender,age,lic_years,pastclaims,exp_years
0,1,WC,White,218,1,29,11,2,0.481862
1,2,FS,White,189,0,27,8,2,0.232717
2,3,GP,Blue,166,1,31,13,0,0.169747
3,4,GP,Yellow,94,0,31,13,0,0.156057
4,5,MP,Blue,308,0,33,15,0,0.427105


In [17]:
new_df = pd.get_dummies(new_df, columns=["province", "carcolour"], drop_first=True)
bool_cols = new_df.select_dtypes(include=["bool"]).columns
new_df[bool_cols] = df[bool_cols].astype(int)

In [18]:
new_df.head()

,id,hp,gender,age,lic_years,pastclaims,exp_years,province_FS,province_GP,province_KZN,...,province_MP,province_NC,province_NW,province_WC,carcolour_Blue,carcolour_Green,carcolour_Red,carcolour_Silver,carcolour_White,carcolour_Yellow
0,1,218,1,29,11,2,0.481862,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,2,189,0,27,8,2,0.232717,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,3,166,1,31,13,0,0.169747,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,4,94,0,31,13,0,0.156057,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,308,0,33,15,0,0.427105,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
X_new = new_df.drop(columns=["id", "exp_years"])
X_new = sm.add_constant(X_new)

In [20]:
X.head()

,const,hp,gender,age,lic_years,pastclaims,province_FS,province_GP,province_KZN,province_LIM,province_MP,province_NC,province_NW,province_WC,carcolour_Blue,carcolour_Green,carcolour_Red,carcolour_Silver,carcolour_White,carcolour_Yellow
0,1.0,74,0,18,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
1,1.0,283,0,81,63,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,1.0,122,0,54,35,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
3,1.0,172,0,39,21,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,1.0,161,0,54,35,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [21]:
prediction = model.predict(X_new, offset=np.log(new_df["exp_years"]))

c:\Users\Armandt\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [22]:
prediction 

0        0.137353
1        0.056276
2        0.035942
3        0.023374
4        0.113807
           ...   
14547    0.040070
14548    0.048379
14549    0.069917
14550    0.043779
14551    0.004666
Name: exp_years, Length: 14552, dtype: float64

In [23]:
annual_frequency = prediction / new_df["exp_years"]


In [24]:
annual_frequency

0        0.285047
1        0.241820
2        0.211741
3        0.149780
4        0.266461
           ...   
14547    0.147832
14548    0.123570
14549    0.290197
14550    0.124923
14551    0.131109
Name: exp_years, Length: 14552, dtype: float64

In [27]:
simulated_claims = np.random.poisson(lam=prediction)
simulated_claims

array([0, 0, 0, ..., 0, 0, 0], shape=(14552,), dtype=int32)

In [34]:
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split

In [35]:
# new

# Response variable
y = df["claims"]

# Predictors (exclude 'id', 'claims', 'exp_years')
X = df.drop(columns=["id", "claims"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
model = PoissonRegressor(alpha=1, fit_intercept=True, max_iter=1000)
model.fit(X_train, y_train, sample_weight=df.loc[y_train.index, "exp_years"])
y_pred = model.predict(X_test)

In [36]:
#from sklearn.metrics import mean_poisson_deviance
    # y_true and y_pred are arrays of true and predicted counts
#mpd = mean_poisson_deviance(y_true, y_pred)
from sklearn.metrics import (
    mean_absolute_error,
    mean_poisson_deviance,
    mean_squared_error,
)

In [38]:
y_pred

array([0.20527501, 0.12798628, 0.21128533, ..., 0.1834419 , 0.12324621,
       0.07873606], shape=(16000,))

In [41]:
mean_absolute_error(y_test, y_pred)


0.2579646637688003

In [42]:
mean_squared_error(y_test, y_pred)


0.1475550255297249

In [43]:
mean_poisson_deviance(y_test, y_pred)

0.575143575686425